In [34]:
import json
from web3 import Web3
from taxes_abi import *

In [35]:
ganache_url = 'HTTP://127.0.0.1:7545'

In [36]:
web3 = Web3(Web3.HTTPProvider(ganache_url))

In [37]:
web3.eth.defaultAccount = web3.eth.accounts[0] 
print(web3.isConnected())

True


In [38]:
contract_abi = json.loads(abi)
contract_bytecode = bytecode

In [40]:
def deploy_contract(abi, bytecode):
    """
    Deploy the contract using Python and web3.py without relying on Remix
    
    Parameters
    --------------
    abi : abi of the contract
    bytecode : bytecode of the contract
    
    Returns
    --------------
    Address where the contract has been deployed
    """
    
    contract = web3.eth.contract(abi=abi, bytecode=bytecode)
    tx_hash = contract.constructor().transact()
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    return tx_receipt.contractAddress


In [41]:
contract_address = deploy_contract(abi=contract_abi, bytecode=contract_bytecode)

print(contract_address)

0x3D9E53dF42D5b202f72C68F0D2E8E5e69Dc0E0c9


In [42]:
# Define contract
rubbish_contract = web3.eth.contract(address=contract_address, abi=contract_abi)

In [43]:
roles = ['municipality'] + [f'citizen_{i}' for i in range(1,6)] + [f'truck_{i}' for i in range(1,3)] + [f'station_{i}' for i in range(1,3)]    
address_dict = {roles[i]: web3.eth.accounts[i] for i in range(len(roles))}

In [44]:
address_dict

{'municipality': '0xC3c932a2dD9bC0fbc5246277d5F19d444E3684d5',
 'citizen_1': '0x69668C6a5934950e912CF99e1af944350b5403f6',
 'citizen_2': '0x44598658a840bceEfc25c292e770D8146f5610Ab',
 'citizen_3': '0x3D70fbcD063E392a90BE463d9adab387AB7347eC',
 'citizen_4': '0x2C98A5B9214B4E4180959d476864d626a0a0C87a',
 'citizen_5': '0x266FEB6959A225cD07814bA55F849B82d8c2BBF0',
 'truck_1': '0xf6FDe303E411fCf49bB5ec754e0b296411acB0b0',
 'truck_2': '0x507673221dC622011BA3e168B00E75d650F8A634',
 'station_1': '0xee2fc939b4AaF6A18067606b46A01eB1BFcBcA0b',
 'station_2': '0x35A351f83d633eE39d8eeBb9A5d42aa054ddDCaB'}

In [45]:
# All functions in the contract
rubbish_contract.all_functions()

[<Function _addGarbageCollectorRole(address)>,
 <Function _addMunicipalityManager(address)>,
 <Function _removeCitizenRole(address)>,
 <Function _removeGarbageCollectorRole(address)>,
 <Function _removeMunicipalityManagers(address)>,
 <Function addCitizen()>,
 <Function generateTrashBag()>,
 <Function payDeposit()>,
 <Function pickFromBin(address,uint256,uint8)>,
 <Function renounceOwnership()>,
 <Function transferOwnership(address)>,
 <Function didIPayDeposit()>,
 <Function getAmountNonRecyclableWaste()>,
 <Function getAmountRecyclableWaste()>,
 <Function getTaxesDue()>,
 <Function getTotalTaxesPaid()>,
 <Function owner()>]

Add citizens to the citizen list using function `addCitizen()`

In [46]:
for citizen in [f'citizen_{i}' for i in range(1,6)]:
    print(citizen)
    tx_hash = rubbish_contract.functions.addCitizen().transact({'from': address_dict[citizen]})

citizen_1
citizen_2
citizen_3
citizen_4
citizen_5


In [47]:
def get_info_from_citizen(contract, citizen):
    """
    Given a contract and a citizen, retrieve information regarding taxes paid/due and rubbish produced
    
    citizen must be a str
    
    Taxes paid -> only municiplaity manager can access this!
    """
    
    from_dict = {'from': address_dict[citizen]}
    paid_deposit = contract.functions.didIPayDeposit().call(from_dict)
    nonrec_waste_amount = contract.functions.getAmountNonRecyclableWaste().call(from_dict)
    rec_waste_amount = contract.functions.getAmountRecyclableWaste().call(from_dict)
    taxes_due = contract.functions.getTaxesDue().call(from_dict)
    #taxes_paid = contract.functions.getTotalTaxesPaid().call(from_dict)
    return paid_deposit, nonrec_waste_amount, rec_waste_amount, taxes_due, #taxes_paid

In [48]:
c1 = get_info_from_citizen(rubbish_contract, 'citizen_1')
c1

(False, 0, 0, 0)

In [49]:
# Minimum deposit
deposit = 100000000000000000
tx_hash = rubbish_contract.functions.payDeposit().transact({'from': address_dict['citizen_1'], 
                                                                   'value': deposit})

In [50]:
# Citizen 1 generates a trash bag 
tx_hash = rubbish_contract.functions.generateTrashBag().transact({'from': address_dict['citizen_1']})

In [51]:
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)

In [52]:
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x44a41804ec9f6d4504f981ca81ee069989510720ea7bd7da786d25809c689630'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x07e5b1952a5e0431c7a4419e66c3a2ee29ff71fb3ae2d64b6f76a72b99aa6c7e'),
 'blockNumber': 8,
 'from': '0x69668C6a5934950e912CF99e1af944350b5403f6',
 'to': '0x3D9E53dF42D5b202f72C68F0D2E8E5e69Dc0E0c9',
 'gasUsed': 26129,
 'cumulativeGasUsed': 26129,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x44a41804ec9f6d4504f981ca81ee069989510720ea7bd7da786d25809c689630'),
   'blockHash': HexBytes('0x07e5b1952a5e0431c7a4419e66c3a2ee29ff71fb3ae2d64b6f76a72b99aa6c7e'),
   'blockNumber': 8,
   'address': '0x3D9E53dF42D5b202f72C68F0D2E8E5e69Dc0E0c9',
   'data': '0xa230cb8fb52cb660380a123d811fd29af7dfe2f2e0e198e1653572cd560230ba00000000000000000000000069668c6a5934950e912cf99e1af944350b5403f6000000000000000000000000000000000000000000000000000000005fcfb43d',
   'topics': 

In [122]:
import time

def handle_event(event):
    print(event)

def log_loop(event_filter, poll_interval):
    while True:
        for event in event_filter.get_new_entries():
            handle_event(event)
        time.sleep(poll_interval)

def main():
    block_filter = web3.eth.filter('latest')
    log_loop(block_filter, 2)

In [123]:
main()

b'1\xb2\x9ayS\xe7\xf5\x9e\x96b\xdeRn^X\xc4\xb5M\xe0\xfe|L\xa9\xab3G\x02dA\xa9\xbaW'


KeyboardInterrupt: 